# Classificação - Base exame diabetes

In [1]:
import pandas as pd


df_train = pd.read_csv("data/diabete/Dados_Treino.csv")

In [3]:
# df_steam.dtypes

df_train.describe()

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,117141.000000,117141.000000,112653.000000,113106.000000,117141.000000,115286.000000,117141.000000,117141.000000,117141.0,114021.000000,...,15108.000000,15108.000000,117141.000000,117141.000000,117141.000000,117141.000000,117141.000000,117141.000000,117141.000000,117141.000000
mean,213013.979853,106.220546,62.009365,29.120761,0.190104,169.603092,662.935847,0.842093,0.0,83.805420,...,246.657985,238.915427,0.001016,0.016126,0.013659,0.025696,0.007222,0.004234,0.020992,0.216269
std,38128.777647,63.483503,16.825337,8.281204,0.392385,10.835608,304.328918,2.504553,0.0,24.985485,...,131.006351,128.170607,0.031857,0.125960,0.116070,0.158226,0.084676,0.064933,0.143357,0.411702
min,147001.000000,1.000000,0.000000,14.844926,0.000000,137.200000,82.000000,-0.250000,0.0,38.600000,...,42.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,179951.000000,50.000000,52.000000,23.596939,0.000000,162.500000,428.000000,0.045833,0.0,66.500000,...,143.333333,137.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,213051.000000,112.000000,64.000000,27.570357,0.000000,170.100000,653.000000,0.155556,0.0,80.000000,...,227.500000,218.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,246063.000000,165.000000,75.000000,32.824772,0.000000,177.800000,970.000000,0.425000,0.0,96.820000,...,332.000000,323.809524,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,279000.000000,204.000000,89.000000,67.814990,1.000000,195.590000,1111.000000,175.627778,0.0,186.000000,...,720.000000,654.813793,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
print("bb")

bb
